<a href="https://colab.research.google.com/github/Search-Prem/E-Waste-Generation-Classification/blob/main/E_Waste_Generation_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E-Waste Classification with EfficientNetV2B0

This notebook trains a convolutional neural network (CNN) using the EfficientNetV2B0 architecture to classify images of e-waste into 10 categories. It includes steps for data loading, preprocessing, model building, training, evaluation, and creating a Gradio interface for inference.

## Setup and Imports

This cell installs the necessary libraries and imports them for use in the notebook.
- `tensorflow`: The main deep learning library.
- `scikit-learn`: Used for calculating class weights.
- `matplotlib`: Used for plotting the training history.
- `gradio`: Used to create a web interface for the model.
- `PIL`: Used for image manipulation within the Gradio interface.

In [ ]:
# Install required packages
!pip install tensorflow scikit-learn matplotlib gradio

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import EfficientNetV2B0
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, classification_report
import gradio as gr
from PIL import Image

## Model Definition and Compilation

This section defines the CNN model. It uses a pre-trained EfficientNetV2B0 model as the base, adds data augmentation layers, global average pooling, a dense layer, dropout, and a final dense layer for classification. The base model's first 100 layers are frozen during initial training. The model is compiled with the Adam optimizer, sparse categorical crossentropy loss, and accuracy as the metric.

In [ ]:
# Enhanced data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.15),
    layers.RandomContrast(0.1),
])

# Load pre-trained model
base_model = EfficientNetV2B0(
    input_shape=(128, 128, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze first 100 layers
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

# Build model with extra dense layer - Removed data_augmentation here
model = tf.keras.Sequential([
    layers.Input(shape=(128, 128, 3)),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),  # New layer added
    layers.Dropout(0.3),                  # Increased dropout
    layers.Dense(10, activation='softmax')
])

# Compile with lower initial learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

## Data Loading and Preparation

This section defines the paths to the training, validation, and test datasets and loads them using `tf.keras.utils.image_dataset_from_directory`. It also calculates class weights to handle potential class imbalance in the dataset.

In [ ]:
# Set paths to your dataset - Update these paths based on where your data is in Colab
# For example, if your data is in Google Drive, you might use:
# from google.colab import drive
# drive.mount('/content/drive')
# train_path = '/content/drive/My Drive/E waste data/modified-dataset/train'
# test_path = '/content/drive/My Drive/E waste data/modified-dataset/test'
# val_path = '/content/drive/My Drive/E waste data/modified-dataset/val'
# Or if you've uploaded your data directly to the Colab environment:
# train_path = '/content/modified-dataset/train'
# test_path = '/content/modified-dataset/test'
# val_path = '/content/modified-dataset/val'

# Replace the following lines with the correct paths for your Colab environment
# Assuming the zip file extracted to a directory named 'modified-dataset' in /content/
train_path = '/content/modified-dataset/train'
test_path = '/content/modified-dataset/test'
val_path = '/content/modified-dataset/val'


# Load datasets
train_data = tf.keras.utils.image_dataset_from_directory(
    train_path,
    shuffle=True,
    image_size=(128, 128),
    batch_size=32
)

val_data = tf.keras.utils.image_dataset_from_directory(
    val_path,
    shuffle=True,
    image_size=(128, 128),
    batch_size=32
)

test_data = tf.keras.utils.image_dataset_from_directory(
    test_path,
    shuffle=False,
    image_size=(128, 128),
    batch_size=32
)

# Get class names
class_names = train_data.class_names
print("Class names:", class_names)

# Calculate class weights for imbalanced data
y_train = np.concatenate([y.numpy() for _, y in train_data], axis=0)
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

# Apply data augmentation to the training dataset
train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))

## Model Training

This section defines callbacks for early stopping and learning rate reduction, and then trains the model using the training data and validation data. The class weights are used to address the imbalanced dataset.

In [ ]:
# Callbacks
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

lr_scheduler = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-6
)

# Train for more epochs (early stopping will prevent overtraining)
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    class_weight=class_weights,
    callbacks=[early_stopping, lr_scheduler]
)

## Evaluation and Visualization

This section evaluates the trained model on the test set and prints the test accuracy and loss. It also plots the training and validation accuracy and loss over epochs to visualize the training progress.

In [ ]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_data)
print(f"\nTest accuracy: {test_acc:.4f}, Test loss: {test_loss:.4f}")

# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy Over Epochs')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss Over Epochs')
plt.show()

## Model Saving and Gradio Interface

This section saves the trained model to a file. It then defines a Gradio interface for classifying new images. The `classify_image` function preprocesses the input image, makes a prediction using the trained model, and returns the predicted class and confidence.

In [ ]:
# Save model
model.save('efficientnet_ewaste_classifier.keras')

# Gradio interface
def classify_image(img):
    img = img.resize((128, 128))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)

    prediction = model.predict(img_array)
    predicted_class = class_names[np.argmax(prediction)]
    confidence = np.max(prediction)

    return f"Predicted: {predicted_class} (Confidence: {confidence:.2f})"

iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    examples=[["sample_battery.jpg"], ["sample_keyboard.jpg"]],
    title="E-Waste Classifier",
    description="Upload an image of e-waste to classify it into one of 10 categories."
)

iface.launch()